In [ ]:

# Tarefa 3 

import math
import pandas as pd
import matplotlib.pyplot as plt

# Função e derivada segunda
def f(x): return math.exp(x)
def f2(x): return math.exp(x)

# ---------------------------------------------------------
# Spline cúbico Que usamos na Tarefa 1
# -----------------------------------------------------------------
def spline_cubico(x, y, ydd0, yddn):
    n = len(x)
    h = [x[i+1]-x[i] for i in range(n-1)]

    lam = [0] + [h[i]/(h[i-1]+h[i]) for i in range(1, n-1)] + [0]
    mu  = [0] + [h[i-1]/(h[i-1]+h[i]) for i in range(1, n-1)] + [0]

    d = [0]*n
    d[0], d[-1] = 2*ydd0, 2*yddn
    for i in range(1, n-1):
        d[i] = 6/(h[i-1]+h[i]) * ((y[i+1]-y[i])/h[i] - (y[i]-y[i-1])/h[i-1])

    # matriz T (tridiagonal)
    T = [[0]*n for _ in range(n)]
    for i in range(n):
        T[i][i] = 2
        if i>0: T[i][i-1] = mu[i]
        if i<n-1: T[i][i+1] = lam[i]

    # eliminação de gauss
    for j in range(n-1):
        if T[j][j]==0: continue
        for i in range(j+1,n):
            fator = -T[i][j]/T[j][j]
            for k in range(j,n):
                T[i][k] += fator*T[j][k]
            d[i] += fator*d[j]

    # retrossubstituição
    M = [0]*n
    for i in range(n-1,-1,-1):
        soma = sum(T[i][k]*M[k] for k in range(i+1,n))
        M[i] = (d[i]-soma)/T[i][i]

    # A e B de cada intervalo
    A, B = [], []
    for i in range(n-1):
        Ai = (y[i+1]-y[i])/h[i] - (M[i+1]-M[i])*h[i]/6
        Bi = y[i] - M[i]*h[i]**2/6
        A.append(Ai)
        B.append(Bi)

    return M, A, B, h

 # Agora com os valores M,A,B,h, definimos totalmente o spline


# --------------------------------------------------------------
# Avaliação do spline (Tarefa 2)
# ---------------------------------------------------------
def spline_eval(x, M, A, B, h, x0):   # busca binária simples pra achar o intervalo [x[i], x[i+1]]
    i0, i1 = 0, len(x)-1
    while i1 - i0 > 1:
        meio = (i0+i1)//2
        if x0 >= x[meio]: i0 = meio
        else: i1 = meio
    xi, xi1 = x[i0], x[i0+1]
    hi = h[i0]
    return ( M[i0]*(xi1-x0)**3 + M[i0+1]*(x0-xi)**3 )/(6*hi) + A[i0]*(x0-xi) + B[i0]  #

# ----------------------------------------------------------
# Funçãoo parar medir o erro máximo 
# ---------------------------------------------------------------
def erro_maximo(n, caso):
    a, b = 0, 1
    x = [a + i*(b-a)/n for i in range(n+1)]
    y = [f(xx) for xx in x]
 
    if caso==1:
        ydd0, yddn = 0, 0
    else:
        ydd0, yddn = f2(a), f2(b)

    M,A,B,h = spline_cubico(x,y,ydd0,yddn)

    m = 10117
    passo = (b-a)/m
    emax = 0
    xx = a
    for _ in range(m+1):
        s = spline_eval(x,M,A,B,h,xx)
        err = abs(f(xx)-s)
        if err > emax: emax = err
        xx += passo
    return emax

# ---------------------------------------------------------
# Loop para Calcular num Range de 10 a 200 com intervalo der 10
# ---------------------------------------------------------
valores = []
for n in range(10, 201, 10):
    e1 = erro_maximo(n,1)
    e2 = erro_maximo(n,2)
    valores.append([n, e1, e2])
    print(f"n={n:3d} -> E1={e1:.3e}  E2={e2:.3e}")

df = pd.DataFrame(valores, columns=["n","E1","E2"])
print("\nTabela final (E1 = caso 1, E2 = caso 2):")
display(df)






n= 10 -> E1=1.333e-03  E2=1.741e-06
n= 20 -> E1=3.335e-04  E2=1.100e-07
n= 30 -> E1=1.483e-04  E2=2.182e-08
n= 40 -> E1=8.340e-05  E2=6.915e-09
n= 50 -> E1=5.338e-05  E2=2.836e-09
n= 60 -> E1=3.707e-05  E2=1.368e-09
n= 70 -> E1=2.723e-05  E2=7.390e-10
n= 80 -> E1=2.085e-05  E2=4.334e-10
n= 90 -> E1=1.647e-05  E2=2.706e-10
n=100 -> E1=1.334e-05  E2=1.776e-10
n=110 -> E1=1.103e-05  E2=1.213e-10
n=120 -> E1=9.267e-06  E2=8.568e-11
n=130 -> E1=7.895e-06  E2=6.221e-11
n=140 -> E1=6.807e-06  E2=4.626e-11
n=150 -> E1=5.930e-06  E2=3.511e-11
n=160 -> E1=5.213e-06  E2=2.712e-11
n=170 -> E1=4.617e-06  E2=2.128e-11
n=180 -> E1=4.118e-06  E2=1.693e-11
n=190 -> E1=3.696e-06  E2=1.364e-11
n=200 -> E1=3.336e-06  E2=1.111e-11

Tabela final (E1 = caso 1, E2 = caso 2):


,n,E1,E2
0,10,0.001333,1.740932e-06
1,20,0.000334,1.100413e-07
2,30,0.000148,2.181655e-08
3,40,0.000083,6.915375e-09
4,50,0.000053,2.835614e-09
5,60,0.000037,1.368441e-09
6,70,0.000027,7.390448e-10
7,80,0.000021,4.333813e-10
8,90,0.000016,2.706222e-10
9,100,0.000013,1.775975e-10
